## Installing and importing the necessary libraries

In [ ]:
!pip install streamlit pyngrok google-generativeai

from pyngrok import ngrok
import os
import threading
import time
import webbrowser

## Setting API

In [ ]:
# Set your ngrok token
#ngrok.set_auth_token("")  # ← Replace with your token

# Set Gemini API Key
#os.environ["GEMINI_API_KEY"] = ""  # ← Replace with your key

## App Code

In [ ]:
app_code = '''
import streamlit as st
import os
from google import genai
from google.genai import types

# Initialize Gemini client
client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))

def main():
    st.set_page_config(page_title="Gemini QA App", layout="wide")
    st.title("📄 Document Q&A with Gemini + Web Search")

    uploaded_file = st.file_uploader("Upload your document (.txt, .pdf, .docx):", type=["txt", "pdf", "docx"])
    question = st.text_input("Ask a question about the document")

    if st.button("Submit") and uploaded_file and question:
        with st.spinner("Uploading document and generating response..."):
            temp_path = os.path.join("temp", uploaded_file.name)
            os.makedirs("temp", exist_ok=True)
            with open(temp_path, "wb") as f:
                f.write(uploaded_file.getbuffer())

            uploaded_gemini_file = client.files.upload(file=temp_path)

            contents = [
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_uri(
                            file_uri=uploaded_gemini_file.uri,
                            mime_type=uploaded_gemini_file.mime_type,
                        ),
                        types.Part.from_text(text=question),
                        types.Part.from_text(
                            text="Please search the web for additional information or context. Always Provide the links of web pages that you visited to give you response."
                        ),
                    ],
                )
            ]

            tools = [types.Tool(google_search=types.GoogleSearch())]
            generate_config = types.GenerateContentConfig(
                tools=tools,
                response_mime_type="text/plain",
            )

            response_text = ""
            for chunk in client.models.generate_content_stream(
                model="gemini-2.0-flash",
                contents=contents,
                config=generate_config,
            ):
                response_text += chunk.text

            st.subheader("💡 Answer")
            st.markdown(response_text)

            os.remove(temp_path)

if __name__ == "__main__":
    main()
'''

## Running the app

In [ ]:
# Save the app code to a Python file
with open("streamlit_app.py", "w") as f:
    f.write(app_code)

def run_app():
    os.system("streamlit run streamlit_app.py")

# Start Streamlit in a new thread
threading.Thread(target=run_app).start()

# Wait for Streamlit to boot
time.sleep(5)

# Expose the app via ngrok
public_url = ngrok.connect(8501)
print("🔗 Your Streamlit app is live at:", public_url)
webbrowser.open(public_url)